In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# !mkdir -p /content/projects/oil-bottle/data/labeled
# !unzip "/content/drive/MyDrive/Nommas/projects/SaudiCanCompany/oil-bottles/data/oilbottle-aziz-raw-data/cropped.zip" -d /content/projects/oil-bottle/data/labeled

# make sure the folder is called train/
# !mv /content/projects/oil-bottle/data/labeled/cropped /content/projects/oil-bottle/data/labeled/train

In [ ]:
%cd drive/MyDrive/eff

In [ ]:
#%%

import os

import tensorflow as tf
from tensorflow.keras import mixed_precision
print(tf.__version__)


from tensorflow.python.compiler.tensorrt import trt

from tensorflow.keras import mixed_precision

# policy = mixed_precision.experimental.Policy('mixed_float16')
# mixed_precision.experimental.set_policy(policy)

#%%

import os
from tqdm.auto import tqdm
import datetime

nowstr = datetime.datetime.now().strftime("%Y-%m-%d-%H.%M.%S")

# data_dir = f"projects/oil-bottle/data/labeled"
data_dir = f"/content/drive/MyDrive/prev_data/Colab Notebooks/Nommas/projects/SaudiCanCompany/oil-bottle/data/oilbottle-aziz-raw-data/cropped"

clf_dir = f"/content/drive/MyDrive/eff/checkpoints/EfficientNetB0_{nowstr}"

os.makedirs(clf_dir, exist_ok=True)

data_dir = os.path.abspath(data_dir)
print('data_dir', data_dir)
print('clf_dir', clf_dir)
assert os.path.isdir(data_dir), f'not a real directory "{data_dir}"'

for root, dirs, files in os.walk(data_dir):
    for file in files:
        path = os.path.join(root, file)
        if file.endswith('.DS_Store') or file.endswith('.ipynb_checkpoints'):

            print("Deleting: {}".format(path))

            if os.remove(path):
                print("Unable to delete!")
            else:
                print("Deleted...")


In [5]:

import classifier

from tensorflow.python.compiler.tensorrt import trt_convert
import numpy as np

import matplotlib.pyplot as plt


In [6]:
# !rm -rf /content/projects/can-top/data/labeled/train/defect-hard-line

In [ ]:



#%%

# !pip3 install tflite_model_maker

#%%

classifier_object = classifier.Classifier(clf_dir, clf_dir, opt_mode='trt')
classifier_object.classifier(
    my_dir=data_dir,
    model_type='EfficientNetB0',
    # select from Mobilenet','MobilenetV2', 'VGG19','InceptionV3', 'ResNet50V2', 'NASNetMobile', 'DenseNet201
    structure=1,
    # 3 for train,test, valid directories, 2 for train, test directories 1 for a single directory dir
    v_split=.1,
    # only valid if structure =1 or 2. Then it is percentage of training images that will be used for validation
    epochs=100,  # number of epochs to run the model for
    freeze=True,
    # if False all layers of the model are trained for epoch epochs, if True layersof basemodel are not trained
    fine_tune_epochs=10,
    # only used when freeze=True, then all model layers are trained for this value epochs after  initial epochs
    height=224,  # height of images to be used by the model
    width=224,  # width of images to be used by the model
    bands=3,  # bands in image 3 for rgb 1 for grey scale
    batch_size=64,  # batch size used in generators
    lr=.001,  # initial learning rate
    patience=10,  # number of epochs with no performance improvement before learning rate is adjusted
    stop_patience=3,
    # number of times learning rate can be adjusted with performance improvement before stopping training
    threshold=.90,
    # float value is training accuracy<threshold adjust lr on accuracy, above threshold adjust on validation loss
    dwell=False,
    # if True model weights are set back to best_weights if performance did not improve for current epoch
    factor=.5,  # float <1 factor to multiply current learning rate by if performance does not improve
    dropout=.1,  # dropout float<1 defines dropout factor
    print_code=50,  # max number of misclassified files to print out
    neurons_a=1024,
    # number of neurons in dense layer between base model  and final top classification layer
    metrics=[],
    # create a list of desired metrics Note 'accuracy' metric is automatically added to the list of metrics
    visualize=False,
    model_saving_path=clf_dir,
    weight_saving_path=clf_dir)



In [ ]:

# !pip install tensorrt
classifier_object.decision_threshold = 0.5
classifier.confusion_matrix = None
probabilities_array, labels = classifier_object.evaluate(
    clf_dir, data_dir,
    edit_threshold=True,
    is_print_info=True,
    decision_threshold=classifier_object.decision_threshold)

logits = probabilities_array.copy()

In [11]:
probabilities_array = logits.copy()

In [ ]:
bands=3
height=224
width=224
batch_size=32
print_code=10
v_split=.10

train_dir = os.path.join(data_dir, 'train')
train_gen, test_gen, valid_gen = classifier_object.make_gens(1, train_dir, train_dir, train_dir, height, width, bands, batch_size, v_split)

idx2class = {y:x for (x,y) in test_gen.class_indices.items()}


In [ ]:

# probabilities_array[:, 2] = (probabilities_array[:, 2] >= classifier_object.decision_threshold).astype(int)
# probabilities_array[:, 1] = (np.invert(probabilities_array[:, 2].astype(bool))).astype(int)

# probabilities_array[:, 0] = (probabilities_array[:, 0] >= classifier_object.decision_threshold).astype(int)
# probabilities_array[:, 1] = (np.invert(probabilities_array[:, 0].astype(bool))).astype(int)

probabilities_array

#%%


plt.gca().axes.get_yaxis().set_visible(False)
#plt.gca().axes.get_xaxis().set_visible(False)
threshold = np.empty(len(probabilities_array[:, 0]))

my_decision_threshold = 0.1
threshold.fill(my_decision_threshold)

probs_dict = dict(zip(np.unique(labels), [[]]* len(np.unique(labels))))
for i in range(len(labels)):
    probs_dict[labels[i]].append( probabilities_array[i,1] )


# probabilities_array[i, labels[i]]
# plt.scatter(probabilities_defect_hard, np.linspace(0, 1, len(probabilities_defect_hard)), c="red", label="defect_easy")

for label in np.unique(labels):
    probs = probs_dict[label]
    
    plt.scatter(probs, np.linspace(0, 1, len(probs)), label=idx2class[label], alpha=0.5)

    # plt.scatter(probs, np.linspace(0, 1, len(probs)), c="red", label="defect_easy")
    # plt.scatter(probabilities_defect_easy, np.linspace(0, 1, len(probabilities_defect_easy)), c="black", label="defect_difficult")
    # plt.scatter(probabilities_normal, np.linspace(0, 1, len(probabilities_normal)), c="blue", label="normal")

# plt.fill_betweenx(np.linspace(0, 1, len(probabilities_array[:, 0])), my_decision_threshold, 1, color="red", alpha=0.3)
# plt.plot(threshold, np.linspace(0, 1, len(probabilities_array[:, 0])), label="Threshold")

plt.legend()
plt.show()


In [ ]:

for i in range(len(labels)):
    probs = probabilities_array[:, probabilities_array[i]>0.5]
    plt.scatter(probs, np.linspace(0, 1, len(probs)))

plt.fill_betweenx(np.linspace(0, 1, len(probabilities_array[:, 0])), my_decision_threshold, 1, color="red", alpha=0.3)
plt.plot(threshold, np.linspace(0, 1, len(probabilities_array[:, 0])), label="Threshold")

plt.legend()
plt.show()


In [39]:
!jupyter nbconvert --ClearOutputPreprocessor.enabled=True --inplace nommas_defect_detection_classifier.ipynb


[NbConvertApp] Converting notebook nommas_defect_detection_classifier.ipynb to notebook
[NbConvertApp] Writing 46015 bytes to nommas_defect_detection_classifier.ipynb
